<a href="https://colab.research.google.com/github/arshiyasaiyad/Calculator/blob/main/CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import keras.backend as K #imp
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
import tensorflow as tf
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
import gensim
data=open(r"/content/corona.txt")
data
corona_data = [text for text in data if text.count(' ') >= 2]
corona_data
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data=vectorize.texts_to_sequences(corona_data)
total_vocab = sum(len(s) for s in corona_data)
word_count=len(vectorize.index_word)+1
corona_data
print(total_vocab)
print(word_count)
window_size = 2
# Defining utility to generate context word pairs
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
#         print("zero",text)
        for idx, word in enumerate(text):
#             print("first",idx,word)
            context_word = []
            target   = []
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([
                text[i]
                for i in range(begin, end)
                if 0 <= i < text_len
                and i != idx
            ])
            target.append(word)
#             print("second",context_word,target)
            contextual = pad_sequences(
                context_word,
                maxlen=total_length
            )
            final_target = tf.keras.utils.to_categorical(
                target,
                total_vocab
            )
#             print("third",contextual,final_target)
            yield(contextual, final_target)
# Defining the model architecture
model = Sequential()
model.add(
    Embedding(
        input_dim=total_vocab,
        output_dim=100,
        input_length=window_size*2
    )
)
model.add(
    Lambda(
        lambda x: K.mean(x, axis=1),
        output_shape=(100,)
    )
)
model.add(
    Dense(
        total_vocab,
        activation='softmax'
    )
)
model.summary()
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam'
)
for i in range(10):
    cost = 0
    for x, y in cbow_model(corona_data, window_size, total_vocab):
        cost += model.train_on_batch(x, y)
    print("Epoch ", i,"\t: ", cost)
dimensions = 100
vect_file = open('./vectors.txt','w')
vect_file.write('{} {}\n'.format(102, dimensions))
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()
cbow_output = gensim.models.KeyedVectors.load_word2vec_format(
    'vectors.txt',
    binary=False
)
cbow_output.most_similar(positive=['speed'])

198
103
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            19800     
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 198)               19998     
                                                                 
Total params: 39798 (155.46 KB)
Trainable params: 39798 (155.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch  0 	:  1041.5042796134949
Epoch  1 	:  989.8544821739197
Epoch  2 	:  899.150280714035
Epoch  3 	:  824.4052407741547
Epoch  4 	:  772.6653254032135
Epoch  5 	:  722.9558879137039
Epoch  6 	:  670.8583127260208
Epoch  7 	:  617.1934343576431
Epoch  8 	:  563.4551504

[('–transmission', 0.8506500720977783),
 ('difference', 0.7564032077789307),
 ('number', 0.7507978081703186),
 ('driver', 0.6726295351982117),
 ('before', 0.671950101852417),
 ('reproductive', 0.6712654829025269),
 ('serial', 0.6571751236915588),
 ('appearance', 0.6363546252250671),
 ('interval', 0.6311798691749573),
 ('virus', 0.6108498573303223)]